In [2]:
from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types

# Short audio file

In [2]:
def transcribe_gcs(gcs_uri):
    """Transcribes the audio file specified by the gcs_uri."""
    client = speech.SpeechClient()
    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        language_code='es-CO')

    response = client.recognize(config, audio)
    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    return response 

#### First I had to convert the audio to Flac using : ffmpeg -i file-2.mp3 -c:v libx264 short_fla.flac
#### Then I changed the channel to ffmpeg -i short_fla.flac -ac 1 mono_short.flac

In [3]:
s = transcribe_gcs("gs://radioscrapping/Radio/mono_short.flac")

In [4]:
s.results[0].alternatives[0].transcript

'Gabriel Jesús y ya se tomó la titularidad del partido con Venezuela Cuando entró perdió la titular el jugador del Ajax richarlison tiene paperas y everton es un extremo izquierdo que es derecho de mucha habilidad grandes diagonales jugador muy desequilibrante'

# Long audio file

In [4]:
def transcribe_gcs(gcs_uri):
    """Asynchronously transcribes the audio file specified by the gcs_uri."""
    client = speech.SpeechClient()

    audio = types.RecognitionAudio(uri=gcs_uri)
    config = types.RecognitionConfig(
        encoding=enums.RecognitionConfig.AudioEncoding.FLAC,
        language_code='es-CO')

    operation = client.long_running_recognize(config, audio)

    print('Waiting for operation to complete...')
    response = operation.result(timeout=90)

    # Each result is for a consecutive portion of the audio. Iterate through
    # them to get the transcripts for the entire audio file.
    return response

In [13]:
ss = transcribe_gcs("gs://radioscrapping/Radio/mono_long.flac")

Waiting for operation to complete...


In [6]:
a = ss.results[0].alternatives[0].transcript

In [7]:
b = ss.results[1].alternatives[0].transcript

In [8]:
c = ss.results[2].alternatives[0].transcript

In [9]:
d = ss.results[3].alternatives[0].transcript

In [12]:
text = a + b + c + d

In [16]:
len(text.split(" "))

606

In [17]:
ss

results {
  alternatives {
    transcript: "acumulaci\303\263n de tarjetas amarillas va a estar Alan y tambi\303\251n confirma que va a estar Arturo el que si perdi\303\263 la titularidad definitivamente arranc\303\263 siendo titular fue richarlison porque la gran figura del equipo brasile\303\261o el que ha hecho olvidar a Neymar por momentos es everton y el otro que arranc\303\263 siendo suplente es Gabriel Jes\303\272s y ya se tom\303\263 la titularidad del partido con Venezuela Cuando entr\303\263 no eres perdi\303\263 la titular el jugador del Ajax richarlison tiene paperas y everton es un extremo izquierdo que es el derecho de mucha habilidad grandes diagonales jugador muy desequilibrante firmino jugando como como 9 y pues ahorita estaremos hablando de lo que es Paraguay ha planteado para hoy un equipo muy rocoso lleno de gente en la mitad de atr\303\241s vamos a ver si al fin sale con 5 defensas parece que si sale con 5 Defensores pues para contener las hordas Alba"
    confiden

In [13]:
from datetime import datetime


dateTimeObj = datetime.utcnow()
timestampStr = dateTimeObj.strftime("%m/%d/%Y, %H:%M:%S")
timestampStr

'08/21/2019, 19:08:18'

In [28]:
from io import BytesIO
from google.cloud import storage
storage_client = storage.Client()

def download_file(request):
    bucket = storage_client.get_bucket('radioscrapping')
    blob = bucket.get_blob('Blue_21-Aug-2019 21:10:02.028598.flac')
    file = BytesIO(blob.download_as_string())
    return file

In [35]:
storage_client = storage.Client()
bucket = storage_client.get_bucket('radioscrapping')
blob = bucket.get_blob('blue/Blue_21-Aug-2019 21:10:02.028598.flac')
file = BytesIO(blob.download_as_string())

In [37]:
a

In [26]:
from pydub import AudioSegment
a = AudioSegment.from_file("mono_long.flac")
a.duration_seconds

266.76244897959185

In [17]:
from datetime import datetime
from google.cloud import bigquery
from pydub import AudioSegment


dateTimeObj = datetime.now()
timestampStr = dateTimeObj.strftime("%Y-%m-%d %H:%M:%S")

ss = transcribe_gcs("gs://radioscrapping/Radio/mono_long.flac")
a = AudioSegment.from_file("mono_long.flac")
a.duration_seconds


array = []
for result in ss.results:
    text = result.alternatives[0].transcript
    array.append(text)
    
def bq_create_table():
    schema = [
    bigquery.SchemaField('source', 'STRING', mode='REQUIRED'),
    bigquery.SchemaField('text_content', 'STRING', mode='REPEATED'),
    bigquery.SchemaField('scrapping_timestamp', 'TIMESTAMP', mode='REQUIRED')
            ]
    
    bigquery_client = bigquery.Client()
    dataset_ref = bigquery_client.dataset('radio_scrapping')
# Prepares a reference to the table
    table_ref = dataset_ref.table('blue')
    try:
        bigquery_client.get_table(table_ref)
    except:
            table = bigquery.Table(table_ref, schema=schema)
            table = bigquery_client.create_table(table)
            print('table {} created.'.format(table.table_id))

def upload_to_bq(row):
            # Instantiates a client
    bigquery_client = bigquery.Client()
    dataset_ref = bigquery_client.dataset('radio_scrapping')
    table_ref = dataset_ref.table('blue')
    table = bigquery_client.get_table(table_ref)
    rows_to_insert = [
            row
    ]
    errors = bigquery_client.insert_rows(table, rows_to_insert)
    print(errors)
    assert errors == []
bq_create_table()
upload_to_bq(["Blue", array, timestampStr])

[]


In [38]:
60*60


3600